<br>
<center>
<font size="+6"><b>Group 3 Project</b></font><br><br>
<font size="+3"><b>Music Imagery Information Retrieval</b></font><br>
</center>

<br><br>

- Students: *Elias Düker, Leon Grund, Jakob Prager, Veronika Pressler*
- Course: *Neuroinformatics: Machine Learning for Neuronal Data Analysis (2022W)*
- Deadline: *January 26, 2023*

<br>

**Todo's.**
- [ ] Abstract 
- [ ] Data
- [ ] Preprocessing
- [ ] Methods &#9888;&#65039;
- [ ] Experiments
- [ ] Results

<br>

**Abstract summary.**
<font color="red">Here some words what we do in the following and a short summary of the results. I guess we don't need a Introduction and Conclusion section in this notebook...</font>

<br>

**Table of contents.**
1. [Raw data](#data)
2. [Preprocessing](#preprocessing)
3. [Classifier](#classifier)
4. [Experiments](#experiments)
5. [Results](#results)

<br>

**Requirements.**
Please make sure that all 10 `fif` files<sup>1</sup> can be found in a folder `data` in the same directory as this notebook, i.e.
```
project/
│   readme.md
│   this_notebook.ipynb
│
└───data/
        P01-raw.fif
        P04-raw.fif
        ...
        P14-raw.fif

```

<sub><sup>1</sup>You can download the raw files here: http://bmi.ssc.uwo.ca/OpenMIIR-RawEEG_v1/.</sub>

In [1]:
import numpy as np
import matplotlib
import sys
import pandas as pd

# mne
import mne
from mne.preprocessing import ICA, read_ica
from mne.decoding import CSP, UnsupervisedSpatialFilter

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [2]:
print('python:', sys.version.split()[0])
print('mne:', mne.__version__)
print('numpy:', np.__version__)
print('matplotlib:', matplotlib.__version__)

python: 3.9.10
mne: 1.2.2
numpy: 1.23.4
matplotlib: 3.6.0


<a id="data"></a>
# 1. Raw data

Below we show an example how to load and illustrate the raw EEG data for subject `P01`.

In [3]:
def get_raw_data(participant_ids=['P01', 'P04', 'P05', 'P06', 'P07', 'P09', 'P11', 'P12', 'P13', 'P14']):
    return [mne.io.read_raw_fif(f'{pid}-raw.fif', verbose=0) for pid in participant_ids]

In [4]:
raw_list = get_raw_data(['P01'])

<a id="preprocessing"></a>
# 2. Preprocessing

The function `preprocess_raw_data` takes care of all preprocessing steps such as band-pass filtering, ICA with EOG detection, and averaging.

In [5]:
def preprocess_raw_data(raw_list, l_freq=1, h_freq=45, ica_components=15):
    
    for raw in raw_list:       
        # band-pass filtering
        raw_filt = raw.copy().load_data().filter(l_freq=l_freq, h_freq=h_freq, phase='zero-double', verbose=0)
        
        # independent component analysis to exclude EOG events
        ica = ICA(n_components=ica_components, max_iter='auto', random_state=97, verbose=0)
        ica.fit(raw_filt, verbose=0)
        eog_indices, eog_scores = ica.find_bads_eog(raw_filt, verbose=0)
        ica.exclude = eog_indices
        raw_ica = ica.apply(raw_filt.load_data(), verbose=0)
        
        # averaging
        data_avg = raw_filt.copy().load_data().set_eeg_reference(ref_channels='average', verbose=0)
        
        yield data_avg
    
    return None

In [6]:
data_list = list(preprocess_raw_data(raw_list))

Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   13.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    4.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

<a id="methods"></a>
# 3. Methods

The implementation is similar to our programming assignment with functions `train_func`, `decision_func`. In addition, I've collected the steps to produce the labelled dataset for different event_id mappings in the function `get_labelled_dataset`. Finally, the `run_pipeline` function applies `CSP`, `StandardScaler` and `LDA` classifier.


In [7]:
def train_func(X_train, y_train):
    clf = Pipeline([
        ("PCA", UnsupervisedSpatialFilter(PCA(10), average=False)),
        ("CSP", CSP(n_components=20, reg='ledoit_wolf', log=True)),
        ("scaler", StandardScaler()),
        ("classify", LinearDiscriminantAnalysis())]) 
    clf.fit(X_train, y_train)
    return clf

In [8]:
def decision_func(trained_clf, X_test):
    y_pred = trained_clf.predict(X_test)
    return y_pred

In [9]:
def get_labelled_dataset(data_list, event_labels):
    xs, ys = [], []
    event_mat_list = [mne.find_events(data, stim_channel='STI 014', verbose=0) for data in data_list]
    epochs = [mne.Epochs(data, events=event_ma, event_id=list(event_labels.keys()), picks='eeg', tmin=-0.1, tmax=3.0, verbose=0) 
              for (data, event_ma) in zip(data_list, event_mat_list)]
    
    for epoch in epochs:
        xs.append(epoch.get_data())
        ys.append(np.array([event_labels.get(event_id) for event_id in epoch.events[:, 2]]))    
    return np.concatenate(xs), np.concatenate(ys)

In [10]:
def run_pipeline(data_list, event_labels):
    X_all, y_all = get_labelled_dataset(data_list, event_labels)
    
    split_obj = StratifiedKFold(n_splits=5, shuffle=True)

    acc = []

    for train_index, test_index in split_obj.split(X_all, y_all):
        # get train-test split
        X_train, X_test = X_all[train_index], X_all[test_index]
        y_train, y_test = y_all[train_index], y_all[test_index]

        # train classifier
        trained_clf = train_func(X_train, y_train)

        # predict labels
        y_pred = decision_func(trained_clf, X_test)

        # append accuracy for this fold
        acc.append(accuracy_score(y_test, y_pred))
    
    return acc

<a id="experiments"></a>
# 4. Experiments

The event markers recorded in the raw EEG comprise:
- Trial labels (as a concatenation of stimulus ID and condition) at the beginning of each trial
- Exact audio onsets for the first cue click of each trial in conditions 1 and 2 (detected by the Stimtracker)
- Subject feedback for the condition 4 trials (separate event IDs for positive and negative feedback)


For example, the event "imagining Chim Chim Cheree with lyrics and cue clicks" would be 12, since
- songs with lyrics belong the the `0`th group
- Chim Chim Cheree is the `1`st song in the group
- imagining with cue clicks is condition `2`

$\Rightarrow$ `012` = 12.

In [11]:
GROUPS = [0, 1, 2]
SONGS = [1, 2, 3, 4]
CONDS = [1, 2, 3, 4]

In [12]:
# lyrics vs non-lyrics
lyrics = [int(f'{g}{s}{c}') for g in [0] for s in SONGS for c in CONDS]
non_lyrics = [int(f'{g}{s}{c}') for g in [1] for s in SONGS for c in CONDS]

mapping_ex1 = {}
mapping_ex1.update({event_id: 0 for event_id in lyrics})
mapping_ex1.update({event_id: 1 for event_id in non_lyrics})

In [15]:
# perception vs imagination (balanced) 1 vs 4
perception14 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [1]]
imagination14 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [4]]

mapping_ex2 = {}
mapping_ex2.update({event_id: 0 for event_id in perception14})
mapping_ex2.update({event_id: 1 for event_id in imagination14})

In [16]:
# perception vs imagination (balanced) using a specific song Chim Chim Cheree id 1

perception_CCCheree = [int(f'{g}{s}{c}') for g in GROUPS for s in [1] for c in [1]]
imagination_CCCheree = [int(f'{g}{s}{c}') for g in GROUPS for s in [1] for c in [2]]

mapping_ex4 = {}
mapping_ex4.update({event_id: 0 for event_id in perception_CCCheree})
mapping_ex4.update({event_id: 1 for event_id in imagination_CCCheree})

In [20]:
# perception vs imagination (balanced) 1 vs 2
perception12 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [1]]
imagination12 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [2]]

mapping_ex5 = {}
mapping_ex5.update({event_id: 0 for event_id in perception12})
mapping_ex5.update({event_id: 1 for event_id in imagination12})

In [21]:
# perception vs imagination (balanced) 1 vs 3
perception13 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [1]]
imagination13 = [int(f'{g}{s}{c}') for g in GROUPS for s in SONGS for c in [3]]

mapping_ex6 = {}
mapping_ex6.update({event_id: 0 for event_id in perception13})
mapping_ex6.update({event_id: 1 for event_id in imagination13})

<a id="Results"></a>
# 5. Results

In [22]:
results = {
    'lyrics vs non-lyrics': run_pipeline(data_list, mapping_ex1),
    'perception vs imagination (balanced) 1 vs 4': run_pipeline(data_list, mapping_ex2),
    'perception vs imagination (balanced) 1 vs 2': run_pipeline(data_list, mapping_ex5),
    'perception vs imagination (balanced) 1 vs 3': run_pipeline(data_list, mapping_ex6),
    'perception vs imagination (balanced) using a specific song: Chim Chim Cheree': run_pipeline(data_list, mapping_ex4)
}

Using data from preloaded Raw for 160 events and 1588 original time points ...
0 bad epochs dropped
Computing rank from data with rank=None
    Using tolerance 1.3e-05 (2.2e-16 eps * 10 dim * 5.8e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Computing rank from data with rank=None
    Using tolerance 1.3e-05 (2.2e-16 eps * 10 dim * 5.7e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Computing rank from data with rank=None
    Using tolerance 1.3e-05 (2.2e-16 eps * 10 dim * 5.7e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Computing rank 

Estimating covariance using LEDOIT_WOLF
Done.
Computing rank from data with rank=None
    Using tolerance 1.2e-05 (2.2e-16 eps * 10 dim * 5.3e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Computing rank from data with rank=None
    Using tolerance 1.1e-05 (2.2e-16 eps * 10 dim * 5.1e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Computing rank from data with rank=None
    Using tolerance 1.2e-05 (2.2e-16 eps * 10 dim * 5.5e+09  max singular value)
    Estimated rank (mag): 10
    MAG: rank 10 computed from 10 data channels with 0 projectors
Reducing data rank from 10 -> 10
Estimating covariance using LEDOIT_WOLF
Done.
Using data from preloaded Raw for 120 events and 1588 original time p

In [23]:
for title, acc in results.items():
    print('#####', title, '#####')
    # print('Accuracies', acc)
    print('Mean', np.mean(acc), '+/-', np.std(acc))
    print()

##### lyrics vs non-lyrics #####
Mean 0.45625 +/- 0.0701560760020114

##### perception vs imagination (balanced) 1 vs 4 #####
Mean 0.8583333333333334 +/- 0.07728015412913088

##### perception vs imagination (balanced) 1 vs 2 #####
Mean 0.5166666666666666 +/- 0.062360956446232366

##### perception vs imagination (balanced) 1 vs 3 #####
Mean 0.55 +/- 0.06666666666666667

##### perception vs imagination (balanced) using a specific song: Chim Chim Cheree #####
Mean 0.4333333333333333 +/- 0.1699673171197595

